In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
! git clone https://www.github.com/pafitis/thesis
# drive.mount('/content/drive')
% cd thesis

Cloning into 'thesis'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 173 (delta 82), reused 138 (delta 49), pack-reused 0
Receiving objects: 100% (173/173), 16.60 MiB | 28.51 MiB/s, done.
Resolving deltas: 100% (82/82), done.
/content/thesis


In [5]:
! pip install spacy-nightly

     |████████████████████████████████| 12.7 MB 162 kB/s 
     |████████████████████████████████| 621 kB 44.7 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 9.1 MB 18.5 MB/s 
     |████████████████████████████████| 456 kB 43.5 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 2.2.4 requires catalogue<1.1.0,>=0.0.7, but you have catalogue 2.0.4 wh

In [6]:
!python -m spacy download en_core_web_trf

2021-07-19 23:01:15.787907: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 459.7 MB 1.2 MB/s 
     |████████████████████████████████| 12.8 MB 32.0 MB/s 
     |████████████████████████████████| 998 kB 51.9 MB/s 
     |████████████████████████████████| 2.2 MB 63.1 MB/s 
     |████████████████████████████████| 3.3 MB 59.7 MB/s 
     |████████████████████████████████| 895 kB 56.2 MB/s 
  Created wheel for en-core-web-trf: filename=en_core_web_trf-3.0.0-py3-none-any.whl size=459708991 sha256=b5d6865114f706d0066ec065c32b07cf0726011ca05bda44fffffbe94345f36b
  Stored in directory: /tmp/pip-ephem-wheel-cache-19ubvyc6/wheels/be/d9/92/c65214265163256ecc7588e6468c43feb86424a69bfe2fddb9
Successfully built en-core-web-trf
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
✔ Download and installa

In [7]:
! pip install pickle5

     |████████████████████████████████| 132 kB 33.0 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219300 sha256=c89aaf9511588b0096b7e8e94277f52309ce225ec22f2e7d5ebe2eaed4fbc19d
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [8]:
import sys
sys.path.append('thesis')
# sys.path.append('/content/thesis')
# sys.path.append('/content/drive/MyDrive/thesis')

In [9]:
import pandas as pd
import numpy as np
import logging, spacy, sys, os, json, requests
import pickle5 as pickle
import matplotlib.pyplot as plt

from helpers.classes import Collection
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

from helpers.cloze_generation import \
    generate_clozes_from_point, named_entity_answer_generator as ne_answer_generator, noun_phrase_answer_generator as np_answer_generator

from helpers.language_modelling import \
    find_nth_substring, multitoken_prediction, check_model, run_language_model

In [10]:
with open('pickles/dataset_20210625_184837.pkl', 'rb') as f:
    df = pickle.load(f)

In [11]:
df.head()

,bulletin,type,point,data
0,businessindustryandtrade/business/businessserv...,date_and_percent,"In 2019, approximate gross value added at basi...",[/businessindustryandtrade/business/businessse...
1,businessindustryandtrade/business/businessserv...,date_and_percent,"The non-financial services sector, which accou...",[/businessindustryandtrade/business/businessse...
2,businessindustryandtrade/business/businessserv...,date_and_percent,Total turnover and purchases of the UK non-fin...,[/businessindustryandtrade/business/businessse...
3,businessindustryandtrade/business/businessserv...,date_and_percent,"Out of the 12 UK regions, 8 regions experience...",[/businessindustryandtrade/business/businessse...
4,businessindustryandtrade/business/businessserv...,date_and_percent,"West Midlands, Yorkshire and The Humber, Scotl...",[/businessindustryandtrade/business/businessse...


# RoBERTa

In [12]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch
from time import time

print('Initialising language model...')
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = RobertaForMaskedLM.from_pretrained('roberta-base').to(device)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# test_df = df[:3]

start_time = time()
print('Running language model...')
results, entity_set, entities = run_language_model(df, model, tokenizer, True)

end_time = time()

print('Done!')
print(f'Time took: {end_time - start_time}')

Initialising language model...


  0%|          | 0/2015 [00:00<?, ?it/s]


Running language model...


100%|██████████| 2015/2015 [42:17<00:00,  1.26s/it]

Done!
Time took: 2537.912506580353


In [14]:
results

[[('2019',
   0.5354322791099548,
   '2019',
   0,
   'ccf75a2c157eacc5253f0e8a55b1ded9f5386d58'),
  ('pounds billion pounds',
   1.6175238641521466,
   '£1,313.9 billion',
   0,
   'fc3346f2545ca5059bdb5150b998300c5001bea5'),
  ('% growth',
   0.8860111998208877,
   '£42.8 billion',
   0,
   '81aa32270d547a368e7b806646c72069ba4fac43'),
  ('aVAA',
   1.648535256871221,
   '3.4%',
   0,
   'f308eb661ad1d6c8021cf9e22d8e8c7d9c5ff50e'),
  ('2018',
   0.8733159303665161,
   '2018',
   0,
   '0a0bcf4b990f1926bc87fa662badadaf7e23d5fa')],
 [],
 [('£ billion billion',
   2.214230118642562,
   '£4,101.5 billion',
   2,
   '0ff2c435e1d192045540e12b44315fa61592a451'),
  ('£,,, billion',
   4.769513084322743,
   '£2,761.4 billion',
   2,
   '1226559ad975ee3d495c004eb469d294d87bfd5d'),
  ('£ billion billion',
   2.753080542539142,
   '£70.5 billion',
   2,
   '8444327426fe4840ba8daa1f5058bba3bee0eb42'),
  ('...%',
   3.2442595547673196,
   '1.7%',
   2,
   'ee292c95bfec63841e1eb11786ced73e918660d2')

- need to sort out how the answer_given / final_answer is sorted because it seems its wrong
- should think about all these repetitive predictions (respectively x3 etc)
- should fix that trailing whitespace before mask